In [1]:
import os
import json
import pandas as pd
import numpy as np
import json
# from google.colab import drive
# drive.mount("/content/gdrive")

In [2]:
import sys
print(sys.version)
!which python

3.7.11 (default, Jul 27 2021, 07:03:16) 
[Clang 10.0.0 ]
/Users/yatingtian/opt/anaconda3/bin/python


#### Helper functions

In [3]:
def convert_jsonl_pd(filepath):
    with open(filepath, 'r') as json_file:
        json_list = list(json_file)

    dat = []
    for json_str in json_list:
        result = json.loads(json_str)
        dat.append(result)

    dat = pd.DataFrame(dat)
    return dat

## Add Hateful Speech Detection Model

From Huggingface
- https://huggingface.co/Hate-speech-CNERG/bert-base-uncased-hatexplain?text=I+like+you.+I+love+you

The model is used for classifying a text as Hatespeech, Normal or Offensive. 


References:
- https://huggingface.co/docs/transformers/task_summary

In [5]:
import os
path = '/Users/yatingtian/nb/omscs/final/'
os.chdir(path)

In [6]:
# check % of hateful memes with specific word
train = convert_jsonl_pd('./vilio/data/train.jsonl')
# np.mean(train.label[train.text.apply(lambda x:x.find('asian') != -1)])

In [7]:
train[train.label==1].head()

,id,img,label,text
7,70914,img/70914.png,1,tattoos are bad for your health i know 5 milli...
10,79351,img/79351.png,1,jew mad? get fuhrerious!
12,25489,img/25489.png,1,brother... a day without a blast is a day wasted
23,97180,img/97180.png,1,if there is no race it can't be homicide
26,15872,img/15872.png,1,when you come home from a long day of suicide ...


In [8]:
# The model is used for classifying a text as Normal or Abusive (Hatespeech and Offensive).
# https://huggingface.co/Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two?text=when+you+finger+an+asian+girl
from transformers import AutoTokenizer, AutoModelForSequenceClassification
### from models.py
from models import *
tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two")
model = Model_Rational_Label.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two")
inputs = tokenizer("He is a great guy", return_tensors="pt")
prediction_logits, _ = model(input_ids=inputs['input_ids'],attention_mask=inputs['attention_mask'])

In [9]:
prediction_logits

tensor([[ 1.8871, -1.3886]], grad_fn=<AddmmBackward>)

In [10]:
torch.softmax(prediction_logits, dim=1)

tensor([[0.9636, 0.0364]], grad_fn=<SoftmaxBackward>)

In [11]:
train_text = [json.loads(jline)['text'] for jline in open('./vilio/data/train.jsonl', "r").read().split('\n')]

In [12]:
batch_sentences = train_text
encoded_inputs = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors='pt')

In [13]:
# https://huggingface.co/Hate-speech-CNERG/bert-base-uncased-hatexplain?text=I+like+you.+I+love+you
# The model is used for classifying a text as Hatespeech, Normal or Offensive. 
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain")
model = AutoModelForSequenceClassification.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain")
inputs = tokenizer("He is a great guy", return_tensors="pt")
outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])

In [59]:
outputs['attentions'][0].shape

torch.Size([1, 12, 7, 7])

In [15]:
import torch
prediction = torch.softmax(outputs['logits'], dim=1)
prediction.tolist()[0]

[0.04230513423681259, 0.7102062702178955, 0.24748849868774414]

### Add prediction output from bert-base-uncased-hatexplain

In [87]:
# Add prediction output from bert-base-uncased-hatexplain
def add_hate_proba(text):
#     """data: Pandas dataframe"""
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
    prediction = torch.softmax(outputs['logits'], dim=1)
    prediction = prediction.tolist()[0]
    return prediction

In [88]:
from tqdm import tqdm
tqdm.pandas()

In [168]:
part = 'dev_seen'
df = convert_jsonl_pd(f'./vilio/data/{part}.jsonl')

In [118]:
%%time
num_output = df.shape[0]
df['output'] = df['text'].iloc[:num_output].progress_apply(add_hate_proba)

100%|███████████████████████████████████████| 2000/2000 [03:39<00:00,  9.09it/s]

CPU times: user 3min 37s, sys: 2.17 s, total: 3min 39s
Wall time: 3min 39s


In [119]:
split_df = pd.DataFrame(df['output'].iloc[:num_output].tolist(), columns=['hatespeech_prob', 'normal_prob', 'offensive_prob'])
split_df.head()

,hatespeech_prob,normal_prob,offensive_prob
0,0.026279,0.753008,0.220712
1,0.084063,0.629737,0.286199
2,0.061047,0.687515,0.251438
3,0.080715,0.218909,0.700377
4,0.049695,0.678734,0.271571


In [120]:
concat_df = pd.concat([df[:num_output], split_df], axis=1)

In [121]:
concat_df.head()

,id,img,label,text,output,hatespeech_prob,normal_prob,offensive_prob
0,15740,img/15740.png,1,when someone tells you how to bbq,"[0.026279054582118988, 0.7530084848403931, 0.2...",0.026279,0.753008,0.220712
1,38794,img/38794.png,1,when they say white folks don't know how to cook,"[0.08406342566013336, 0.6297371983528137, 0.28...",0.084063,0.629737,0.286199
2,60792,img/60792.png,1,the original derp-face,"[0.061047375202178955, 0.6875145435333252, 0.2...",0.061047,0.687515,0.251438
3,71824,img/71824.png,1,okay here you go! you piece of shit!,"[0.08071465790271759, 0.21890877187252045, 0.7...",0.080715,0.218909,0.700377
4,04796,img/04796.png,1,xboxone farming 1619 simulator,"[0.04969502240419388, 0.6787340044975281, 0.27...",0.049695,0.678734,0.271571


In [195]:
import joblib
import pandas as pd
part='test_seen'
concat_df = joblib.load(f'input/{part}_concat.joblib')
concat_df.to_csv(f'input/{part}_concat.csv')

In [123]:
# # delete dataframe to save memory space
# del split_df
# del df

### Check crosstab results

In [226]:
import joblib
import pandas as pd

part='train'
concat_df = joblib.load(f'input/{part}_concat.joblib')

In [227]:
concat_df.shape

(8596, 8)

In [228]:
import numpy as np
concat_df['max_id'] = concat_df.output.apply(np.argmax)

In [229]:
def convert_id_to_text(max_id):
    if max_id == 0: return "Hatespeech"
    elif max_id == 1: return "Normal"
    else: return "Offensive"

In [230]:
concat_df['result'] = concat_df['max_id'].apply(convert_id_to_text)

In [231]:
# Hatespeech-0, Normal-1 or Offensive-2. 
concat_df.head()

,id,img,label,text,output,hatespeech_prob,normal_prob,offensive_prob,max_id,result
0,42953,img/42953.png,0,its their character not their color that matters,"[0.10603009164333344, 0.6264012455940247, 0.26...",0.106030,0.626401,0.267569,1,Normal
1,23058,img/23058.png,0,don't be afraid to love again everyone is not ...,"[0.0399697944521904, 0.6121541857719421, 0.347...",0.039970,0.612154,0.347876,1,Normal
2,13894,img/13894.png,0,putting bows on your pet,"[0.04539760947227478, 0.706222653388977, 0.248...",0.045398,0.706223,0.248380,1,Normal
3,37408,img/37408.png,0,i love everything and everybody! except for sq...,"[0.13384592533111572, 0.6112882494926453, 0.25...",0.133846,0.611288,0.254866,1,Normal
4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h...","[0.04991312697529793, 0.5152072906494141, 0.43...",0.049913,0.515207,0.434880,1,Normal


In [232]:
#train
pd.crosstab(concat_df['result'], concat_df['label'])

label,0,1
result,,
Hatespeech,64,162
Normal,4880,1979
Offensive,637,874


In [233]:
print(1979/(1979+162+874)) #1
print(4880/(4880+64+637)) #0

0.6563847429519072
0.8743952696649346


In [220]:
#dev_unseen
pd.crosstab(concat_df['result'], concat_df['label'])

label,0,1
result,,
Hatespeech,3,3
Normal,304,171
Offensive,33,26


In [214]:
#test_unseen
pd.crosstab(concat_df['result'], concat_df['label'])

label,0,1
result,,
Hatespeech,7,16
Normal,1118,625
Offensive,125,109


In [154]:
# train_id = train_concat.id.unique()
# print(len(train_id))

### Load Feature Set

In [169]:
# Read existing feature set
# "id" in train.jsonl is an integer
try:
    train_id = [json.loads(jline)['id'] for jline in open(f'./vilio/data/{part}.jsonl', "r").read().split('\n')]
except Exception as e:
    print(e)
print(len(train_id))

500


In [170]:
import csv
import sys
csv.field_size_limit(sys.maxsize)
import time
import base64
# ids=train # train id is int
# FIELDNAMES = ["img_id", "img_h", "img_w", "objects_id", "objects_conf",
#               "attrs_id", "attrs_conf", "num_boxes", "boxes", "features"]

# # vgattr_id = []
# with open('./vilio/data/hm_vgattr5050.tsv') as f:
#     reader = csv.DictReader(f, FIELDNAMES, delimiter="\t")
#     # boxes = args.num_features # Same boxes for all
#     for i, item in enumerate(reader):
#         # vgattr_id.append(item["img_id"])
#         if (int(item["img_id"])) not in ids:
#             continue
#         else:
#             print(item)
#             break

In [171]:
# parser.add_argument("--num_features", type=int, default=100, help='How many features we have per img (e.g. 100, 80)')
# boxes = args.num_features # Same boxes for all
# parser.add_argument("--topk", type=int, default=-1, help='For testing only load topk feats from tsv')

TOPK=-1
FIELDNAMES = ["img_id", "img_h", "img_w", "objects_id", "objects_conf",
              "attrs_id", "attrs_conf", "num_boxes", "boxes", "features"]
NUM_FEATURES =100

In [172]:
# Add tokenized input
import json
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain")
train_text = [json.loads(jline)['text'] for jline in open(f'./vilio/data/{part}.jsonl', "r").read().split('\n')]
batch_sentences = train_text
encoded_inputs = tokenizer(batch_sentences, padding='max_length', truncation=True, max_length=2048, return_tensors='pt')

In [173]:
train_id_loc = {}
i = 0
"""
key (int)
value (int)
"""
for jline in open(f'./vilio/data/{part}.jsonl', "r").read().split('\n'):
    train_id_loc[int(json.loads(jline)['id'])] = i
    i+=1

In [174]:
train_id_loc

{8291: 0,
 46971: 1,
 3745: 2,
 83745: 3,
 80243: 4,
 5279: 5,
 1796: 6,
 53046: 7,
 82301: 8,
 31752: 9,
 27635: 10,
 80597: 11,
 45368: 12,
 17963: 13,
 53968: 14,
 10749: 15,
 25149: 16,
 87520: 17,
 89071: 18,
 9563: 19,
 72048: 20,
 49826: 21,
 26453: 22,
 12650: 23,
 2568: 24,
 83954: 25,
 24316: 26,
 62035: 27,
 65342: 28,
 92058: 29,
 58672: 30,
 54069: 31,
 91586: 32,
 87130: 33,
 95640: 34,
 47819: 35,
 59140: 36,
 13647: 37,
 65832: 38,
 57621: 39,
 40982: 40,
 43275: 41,
 12973: 42,
 74350: 43,
 2478: 44,
 51607: 45,
 65403: 46,
 29750: 47,
 26547: 48,
 2143: 49,
 68192: 50,
 52603: 51,
 5349: 52,
 6491: 53,
 74013: 54,
 27485: 55,
 13026: 56,
 17265: 57,
 91405: 58,
 14026: 59,
 18945: 60,
 83920: 61,
 83497: 62,
 4569: 63,
 39607: 64,
 9715: 65,
 1925: 66,
 94813: 67,
 29437: 68,
 95038: 69,
 64891: 70,
 63280: 71,
 17682: 72,
 84510: 73,
 28690: 74,
 3567: 75,
 95830: 76,
 29873: 77,
 70953: 78,
 56124: 79,
 10386: 80,
 19530: 81,
 94185: 82,
 48370: 83,
 74058: 84,
 450

In [178]:
encoded_inputs

{'input_ids': tensor([[  101,  2317,  2111,  ...,     0,     0,     0],
        [  101, 16534,  2012,  ...,     0,     0,     0],
        [  101,  2115,  2344,  ...,     0,     0,     0],
        ...,
        [  101,  2298,  2012,  ...,     0,     0,     0],
        [  101,  7578,  2177,  ...,     0,     0,     0],
        [  101,  1000,  2043,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [179]:
def load_obj_tsv(fname, ids, encoded_inputs, id_loc, concat_df, topk):
    """Load object features from tsv file.
    :param fname: The path to the tsv file.
    :param topk: Only load features for top K images (lines) in the tsv file.
        Will load all the features if topk is either -1 or None.
    :return: A list of image object features where each feature is a dict.
        See FILENAMES above for the keys in the feature dict.
    """
    data = []
    start_time = time.time()
    print("Start to load Faster-RCNN detected objects from %s" % fname)
    with open(fname) as f:
        reader = csv.DictReader(f, FIELDNAMES, delimiter="\t")
        boxes = NUM_FEATURES # Same boxes for all

        for i, item in enumerate(tqdm(reader)):
            
            # Check if id in list of ids to save memory
            if int(item["img_id"]) not in ids:#modified
                continue

            for key in ['img_h', 'img_w', 'num_boxes']:
                item[key] = int(item[key])
            
            boxes = item['num_boxes']
            decode_config = [
                ('objects_id', (boxes, ), np.int64),
                ('objects_conf', (boxes, ), np.float32),
                ('attrs_id', (boxes, ), np.int64),
                ('attrs_conf', (boxes, ), np.float32),
                ('boxes', (boxes, 4), np.float32),
                ('features', (boxes, -1), np.float32),
            ]
            for key, shape, dtype in decode_config:
                item[key] = np.frombuffer(base64.b64decode(item[key]), dtype=dtype)
                try:
                    item[key] = item[key].reshape(shape)
                except:
                    # In 1 out of 10K cases, the shape comes out wrong; We make necessary adjustments
                    shape = list(shape)
                    shape[0] += 1
                    shape = tuple(shape)
                    item[key] = item[key].reshape(shape)  
 
                item[key].setflags(write=False)
            
            # Add tokenized inputs
            loc = id_loc[int(item["img_id"])]
            
            item['features'] = np.vstack([item['features'], encoded_inputs['input_ids'][loc].reshape(1,-1)])
            
            # Add probability
            hatespeech_prob = concat_df[concat_df.id == int(item["img_id"])]['hatespeech_prob'].to_numpy()
            hatespeech_prob = np.pad(hatespeech_prob, (2047, 0), mode='constant')
            
            normal_prob = concat_df[concat_df.id == int(item["img_id"])]['normal_prob'].to_numpy()
            normal_prob = np.pad(normal_prob, (2047, 0), mode='constant')
            
            offensive_prob = concat_df[concat_df.id == int(item["img_id"])]['offensive_prob'].to_numpy()
            offensive_prob = np.pad(offensive_prob, (2047, 0), mode='constant')
            
            item['features'] = np.vstack([item['features'], hatespeech_prob.reshape(1,-1)])
            item['features'] = np.vstack([item['features'], normal_prob.reshape(1,-1)])
            item['features'] = np.vstack([item['features'], offensive_prob.reshape(1,-1)])
            
            data.append(item)
            if topk is not None and len(data) == topk:
                break
    elapsed_time = time.time() - start_time
    print("Loaded %d images in file %s in %d seconds." % (len(data), fname, elapsed_time))
    return data

In [180]:
fname = './vilio/data/hm_vgattr5050.tsv'
concat_df = joblib.load(f'input/{part}_concat.joblib')
data_out = load_obj_tsv(fname, train_id, encoded_inputs, train_id_loc, concat_df, TOPK)

Start to load Faster-RCNN detected objects from ./vilio/data/hm_vgattr5050.tsv


12140it [01:22, 147.82it/s]

Loaded 0 images in file ./vilio/data/hm_vgattr5050.tsv in 82 seconds.


In [187]:
concat_df['id']=concat_df['id'].astype('int')
print(concat_df.dtypes)

id                   int64
img                 object
label                int64
text                object
output              object
hatespeech_prob    float64
normal_prob        float64
offensive_prob     float64
dtype: object


In [166]:
data_out[0]['features'].shape

(54, 2048)

In [130]:
a = np.array([0.1])
np.pad(a, (2047,0), mode='constant').reshape(1,-1).shape

(1, 2048)

## Fine Tune Pre-trained

In [127]:
from datasets import load_dataset

raw_datasets = load_dataset("imdb")

ImportError: cannot import name 'load_dataset' from 'datasets' (unknown location)

In [128]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain")

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)